In [9]:
import random
import ray
from ray.air import session, Checkpoint
from ray.data import DatasetPipeline
from ray.train.torch import TorchTrainer
from ray.air.config import DatasetConfig

In [10]:
def model(batch):
    return len(batch) * 0.1 * random.uniform(0,1)

def train_loop_per_worker():
    # A DatasetPipeline object is returned when `use_stream_api` is set.
    data_shard: DatasetPipeline = session.get_dataset_shard("train")
    
    loss = 0.0
    # Manually iterate over the data 10 times (10 epochs).
    for epoch in data_shard.iter_epochs(10): 
        # for each epoch iterate over batches
        num_batches = 0
        num_epochs = 0
        for batch in epoch.iter_batches():
            num_batches += 1
            num_epochs += 1
            batch_loss = model(batch)
            loss += batch_loss
        loss /= num_batches * 100
        if num_epochs % 2 == 0:
            print(f"Doing some training on epoch: {num_epochs} for batches: {num_batches} and loss over batch: {loss:.3f}")
        session.report({"loss": loss, "epoch": num_epochs}, 
                       checkpoint=Checkpoint.from_dict({"loss": loss, "epoch": num_epochs}))
    # View the stats for performance debugging.
    # print(data_shard.stats())

In [11]:
# Set N = 200 bytes for this toy example. Typically, you'd set N >= 1GiB.
N = 200
train_ds = ray.data.range_tensor(1000)
trainer = TorchTrainer(train_loop_per_worker,
                       scaling_config={"num_workers": 1},
                       datasets={"train": train_ds},
                       dataset_config={"train": DatasetConfig(use_stream_api=True, 
                                                              stream_window_size=N)},
                      )

In [12]:
result = trainer.fit()

Trial name,status,loc,iter,total time (s),loss,epoch,_timestamp
TorchTrainer_518ce_00000,TERMINATED,127.0.0.1:86187,10,2.87596,0.0255333,20,1658009526


(TorchTrainer pid=86187) 2022-07-16 15:12:04,275	INFO dataset.py:3094 -- Created DatasetPipeline with 20 windows: 400b min, 400b max, 400b mean
(TorchTrainer pid=86187) 2022-07-16 15:12:04,276	INFO dataset.py:3103 -- Blocks per window: 1 min, 1 max, 1 mean
(BaseWorkerMixin pid=86194) 2022-07-16 15:12:04,258	INFO config.py:70 -- Setting up process group for: env:// [rank=0, world_size=1]
(BaseWorkerMixin pid=86194) [W ProcessGroupGloo.cpp:715] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
Stage 1:   0%|          | 0/1 [00:00<?, ?it/s]197) 
(PipelineSplitExecutorCoordinator pid=86197) 
  0%|          | 0/1 [00:00<?, ?it/s]pid=86197) 
(PipelineSplitExecutorCoordinator pid=86197) 
Stage 2:   0%|          | 0/1 [00:00<?, ?it/s]) 
(PipelineSplitExecutorCoordinator pid=86197) 
(PipelineSplitExecutorCoordinator pid=86197) 
Stage 2: : 10it [00:00, 98.6

(BaseWorkerMixin pid=86194) Doing some training on epoch: 20 for batches: 20 and loss over batch: 0.030
Result for TorchTrainer_518ce_00000:
  _time_this_iter_s: 0.8639390468597412
  _timestamp: 1658009525
  _training_iteration: 1
  date: 2022-07-16_15-12-05
  done: false
  epoch: 20
  experiment_id: 858e3fea072f4687a81dde9582592f9f
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  loss: 0.029737603763258125
  node_ip: 127.0.0.1
  pid: 86187
  should_checkpoint: true
  time_since_restore: 1.8201608657836914
  time_this_iter_s: 1.8201608657836914
  time_total_s: 1.8201608657836914
  timestamp: 1658009525
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 518ce_00000
  warmup_time: 0.0030679702758789062
  
(BaseWorkerMixin pid=86194) Doing some training on epoch: 20 for batches: 20 and loss over batch: 0.029


(PipelineSplitExecutorCoordinator pid=86197) 
(PipelineSplitExecutorCoordinator pid=86197) 
Stage 2: : 30it [00:00, 156.58it/s]r pid=86197) 
Stage 0: : 32it [00:00, 164.41it/s]ator pid=86197) 
(PipelineSplitExecutorCoordinator pid=86197) 
(PipelineSplitExecutorCoordinator pid=86197) 
Stage 2: : 46it [00:00, 126.08it/s]r pid=86197) 


(BaseWorkerMixin pid=86194) Doing some training on epoch: 20 for batches: 20 and loss over batch: 0.024
(BaseWorkerMixin pid=86194) Doing some training on epoch: 20 for batches: 20 and loss over batch: 0.026


...

(PipelineSplitExecutorCoordinator pid=86197) 
Stage 0: : 125it [00:00, 170.07it/s]tor pid=86197) 
(PipelineSplitExecutorCoordinator pid=86197) 
Stage 2: : 139it [00:00, 178.54it/s] pid=86197) 
(PipelineSplitExecutorCoordinator pid=86197) 
Stage 0: : 145it [00:00, 177.51it/s]tor pid=86197) 
(PipelineSplitExecutorCoordinator pid=86197) 
Stage 2: : 159it [00:00, 183.98it/s] pid=86197) 


(BaseWorkerMixin pid=86194) Doing some training on epoch: 20 for batches: 20 and loss over batch: 0.025
(BaseWorkerMixin pid=86194) Doing some training on epoch: 20 for batches: 20 and loss over batch: 0.026
(BaseWorkerMixin pid=86194) == Pipeline Window 198 ==
(BaseWorkerMixin pid=86194) Stage 1 read->randomize_block_order: 1/1 blocks executed in 0s
(BaseWorkerMixin pid=86194) * Remote wall time: 134.46us min, 134.46us max, 134.46us mean, 134.46us total
(BaseWorkerMixin pid=86194) * Remote cpu time: 135.0us min, 135.0us max, 135.0us mean, 135.0us total
(BaseWorkerMixin pid=86194) * Peak heap memory usage (MiB): 154894336000.0 min, 154894336000.0 max, 154894336000 mean
(BaseWorkerMixin pid=86194) * Output num rows: 50 min, 50 max, 50 mean, 50 total
(BaseWorkerMixin pid=86194) * Output size bytes: 600 min, 600 max, 600 mean, 600 total
(BaseWorkerMixin pid=86194) * Tasks per node: 1 min, 1 max, 1 mean; 1 nodes used
(BaseWorkerMixin pid=86194) 
(BaseWorkerMixin pid=86194) Stage 2 randomiz

(PipelineSplitExecutorCoordinator pid=86197) 
Stage 0: : 166it [00:01, 185.07it/s]tor pid=86197) 
(PipelineSplitExecutorCoordinator pid=86197) 
Stage 2: : 180it [00:01, 190.00it/s] pid=86197) 
(PipelineSplitExecutorCoordinator pid=86197) 
Stage 0: : 186it [00:01, 188.54it/s]tor pid=86197) 
(PipelineSplitExecutorCoordinator pid=86197) 
Stage 2: : 200it [00:01, 192.24it/s] pid=86197) 


Result for TorchTrainer_518ce_00000:
  _time_this_iter_s: 0.1010432243347168
  _timestamp: 1658009526
  _training_iteration: 10
  date: 2022-07-16_15-12-06
  done: true
  epoch: 20
  experiment_id: 858e3fea072f4687a81dde9582592f9f
  experiment_tag: '0'
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 10
  loss: 0.0255332714713642
  node_ip: 127.0.0.1
  pid: 86187
  should_checkpoint: true
  time_since_restore: 2.875962972640991
  time_this_iter_s: 0.09958791732788086
  time_total_s: 2.875962972640991
  timestamp: 1658009526
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 518ce_00000
  warmup_time: 0.0030679702758789062
  


(PipelineSplitExecutorCoordinator pid=86197) /opt/miniconda3/envs/ray-build/lib/python3.8/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(PipelineSplitExecutorCoordinator pid=86197)   warnings.warn('resource_tracker: There appear to be %d '
2022-07-16 15:12:07,056	INFO tune.py:737 -- Total run time: 4.87 seconds (4.75 seconds for the tuning loop).


In [19]:
print(result)

Result(metrics={'loss': 0.022335927990326774, 'epoch': 20, '_timestamp': 1657944200, '_time_this_iter_s': 0.10506796836853027, '_training_iteration': 10, 'time_this_iter_s': 0.10604286193847656, 'should_checkpoint': True, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 10, 'trial_id': '38a86_00000', 'experiment_id': 'be17d5ff543b4e6b8e9df0efbb62c498', 'date': '2022-07-15_21-03-20', 'timestamp': 1657944200, 'time_total_s': 2.79736590385437, 'pid': 64653, 'hostname': 'Juless-MacBook-Pro-16', 'node_ip': '127.0.0.1', 'config': {}, 'time_since_restore': 2.79736590385437, 'timesteps_since_restore': 0, 'iterations_since_restore': 10, 'warmup_time': 0.002788066864013672, 'experiment_tag': '0'}, checkpoint=<ray.air.checkpoint.Checkpoint object at 0x130b2a460>, error=None, log_dir=PosixPath('/Users/jules/ray_results/TorchTrainer_2022-07-15_21-03-16/TorchTrainer_38a86_00000_0_2022-07-15_21-03-17'), metrics_dataframe=       loss  epoch  _timestamp  _time_this_i

In [20]:
ray.shutdown()